In [49]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np

pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
query_obj = Queries('2022-01-01', end_date)
db_obj = DBImport(db_type='cscart')

analytics = db_obj.data_import(query_obj.analytics_query)
brand = db_obj.data_import(query_obj.brand_query)
inventory = db_obj.data_import(query_obj.inventory_query)
category = db_obj.data_import(query_obj.category_query)

db_obj.connection.close() # db connection close

In [3]:
# basic preprocessing
analytics['purchased_ymd'] = pd.to_datetime(analytics.purchased_at).dt.normalize() # 시간 제외한 날짜만

# 판매 관련 확인

- 판매 급상승의 경우, 과거 판매가 된 날짜만을 바탕으로 진행. 즉, 재고가 있지만 판매가 일어나지 않아 **판매량을 0으로 집계하는 과정은 생략 (∵ ① 지나친 저회전 상품에 대해 급상승을 보고싶은 것도 아니고, ② 재고 정보가 부정확할 수도 있음. 무엇보다도, ③ 0을 복구해줌으로서 분모에 위치시킬 경우 왜곡이 발생할 수 있기 때문)**
- 현재 날짜를 기준으로 최근 일주일을 **타겟기간**, 일주일전으로부터 14일 전까지를 **비교기간**으로 설정 → 각 기간 내에서도 OOS로 인해 판매가 일어나지 않았을 경우, 단순 합으로 비교가 어렵기 때문에 주단위 판매량 합이 아니라, **각 기간의 일평균**으로 연산
- 위와같이 설정할 경우, 비교기간동안 판매가 0건인 상품의 경우 inf로 연산되기 때문에 **비교기간동안 판매가 없는 상품**의 경우 별도 처리 필요 → **1로 대체** (∵ 최근 입고 등의 이슈로 최근 판매량이 급상승 한 상품은 주로 추적을 원하기 때문에, 판매수량이 그대로 입력되도록 분모를 1로 설정. 단, 판매량이 어느정도 되어야 재입고 급상승의 의미가 있기 때문에, **타겟기간 최근 판매량에 threshold 설정**)
- 상대적으로 중요도가 떨어지는, **판매량이 낮은 상품들**도 비율상으로 커져보일 수 있음. **비교기간 평균판매량에 threshold 설정**
- 카테고리의 경우, 기존의 category_M, category_S 두 카테고리를 활용하는 것이 아니라, 상품:카테고리의 1:N 매칭이 이루어지도록 카테고리 테이블을 활용하여 한 상품이 여러 카테고리에 중복 집계가 가능하도록 연산
- 브랜드, 카테고리별 결과의 경우 상위 10개의 결과에 해당하는 상품별 상세정보 테이블 추가로 생성

In [4]:
# target and compare dates
target_end = datetime.now().strftime('%Y-%m-%d')
target_start = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
compare_end = (datetime.now() - timedelta(days=8)).strftime('%Y-%m-%d')
compare_start = (datetime.now() - timedelta(days=22)).strftime('%Y-%m-%d')

In [63]:
### aggregation

## 1. 상품별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 2 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('product_id').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('product_id').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

prod_sales = pd.merge(target_sales, compare_sales, on='product_id', how='left')
non_compare = prod_sales[(prod_sales['compare_qty'].isnull())&(prod_sales['target_qty']>=target_qty_threshold)]
prod_sales = pd.concat([prod_sales[prod_sales['compare_qty'].notnull()], non_compare])
prod_sales['growth_rate_daily'] = prod_sales['target_qty'] / prod_sales['compare_qty']
prod_sales = prod_sales.sort_values(by='growth_rate_daily', ascending=False)
prod_sales = prod_sales[prod_sales['compare_qty']>compare_qty_threshold]
prod_sales = prod_sales.reset_index()

# 아래 브랜드, 카테고리의 상품별 상세를 위한 정보 추가한 테이블 생성
prod_sales_detail = pd.merge(prod_sales, brand, on='product_id', how='left')
prod_sales_detail = pd.merge(prod_sales_detail, category, on='product_id', how='left')
prod_sales_detail = pd.merge(prod_sales_detail, analytics[['product_id', 'product_name_kor']].drop_duplicates(), on='product_id', how='left')

In [64]:
## 2. 브랜드별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 2 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = pd.merge(daily_sales, brand, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['brand'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('brand').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('brand').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

brand_sales = pd.merge(target_sales, compare_sales, on='brand', how='left')
non_compare = brand_sales[(brand_sales['compare_qty'].isnull())&(brand_sales['target_qty']>=target_qty_threshold)]
brand_sales = pd.concat([brand_sales[brand_sales['compare_qty'].notnull()], non_compare])
brand_sales['growth_rate_daily'] = brand_sales['target_qty'] / brand_sales['compare_qty']
brand_sales = brand_sales.sort_values(by='growth_rate_daily', ascending=False)
brand_sales = brand_sales[brand_sales['compare_qty']>compare_qty_threshold]
brand_sales = brand_sales.reset_index()

# 브랜드 상위10개 결과에 해당하는 상품별 상세정보
brand_sales_top10 = brand_sales.head(10)
top10_brands = brand_sales_top10['brand'].tolist()
top10_brands_detail = []
for a_brand in top10_brands: # top10의 순서 유지하기 위해 for문 활용
    brand_prod = prod_sales_detail[prod_sales_detail['brand']==a_brand]
    top10_brands_detail.append(brand_prod)
top10_brands_detail = pd.concat(top10_brands_detail, ignore_index=True)
top10_brands_detail = top10_brands_detail[['brand', 'product_id', 'product_name_kor', 'target_qty', 'compare_qty', 'growth_rate_daily']].drop_duplicates()

In [73]:
## 3. 카테고리별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 2 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = pd.merge(daily_sales, category, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['category'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('category').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})

compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('category').agg({'product_qty':'mean', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})

category_sales = pd.merge(target_sales, compare_sales, on='category', how='left')
non_compare = category_sales[(category_sales['compare_qty'].isnull())&(category_sales['target_qty']>=target_qty_threshold)]
category_sales = pd.concat([category_sales[category_sales['compare_qty'].notnull()], non_compare])
category_sales['growth_rate_daily'] = category_sales['target_qty'] / category_sales['compare_qty']
category_sales = category_sales.sort_values(by='growth_rate_daily', ascending=False)
category_sales = category_sales[category_sales['compare_qty']>compare_qty_threshold]
category_sales = category_sales.reset_index()

# 카테고리 상위10개 결과에 해당하는 상품별 상세정보
category_sales_top10 = category_sales.head(10)
top10_categorys = category_sales_top10['category'].tolist()
top10_categorys_detail = []
for a_category in top10_categorys: # top10의 순서 유지하기 위해 for문 활용
    category_prod = prod_sales_detail[prod_sales_detail['category']==a_category]
    top10_categorys_detail.append(category_prod)
top10_categorys_detail = pd.concat(top10_categorys_detail, ignore_index=True)
top10_categorys_detail = top10_categorys_detail[['category', 'product_id', 'product_name_kor', 'target_qty', 'compare_qty', 'growth_rate_daily']].drop_duplicates()